In [31]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import time

from models import FFNN

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
print("Downloading MNIST dataset...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')
X = X.astype(float)
y = y.astype(int)

In [33]:
X = np.array(X)
y = np.array(y)

X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

encoder = OneHotEncoder(sparse_output=False, categories='auto')
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

input_size = 784  # images 28x28 = 784 pixels
hidden_sizes = [64,64,64] 
output_size = 10  

In [34]:
# Model SKLearn
sklearn_model = MLPClassifier(
    hidden_layer_sizes=hidden_sizes,
    activation='relu',
    solver='sgd', 
    learning_rate_init=0.01,
    max_iter=10,
    random_state=42
)

# Training
print("\nTraining sklearn MLPClassifier...")
start_time = time.time()
sklearn_model.fit(X_train, y_train)
sklearn_training_time = time.time() - start_time
print(f"sklearn MLPClassifier training time: {sklearn_training_time:.2f} seconds")

# Prediksi
sklearn_predictions = sklearn_model.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)
print(f"SKLearn Prediction : {sklearn_accuracy}")



Training sklearn MLPClassifier...
sklearn MLPClassifier training time: 7.80 seconds
SKLearn Prediction : 0.9666428571428571


C:\Users\Shafi\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [35]:
# Model FFNN tanpa RMSNorm
ffnn_model = FFNN(
    layer_sizes=[input_size] + hidden_sizes + [output_size],
    activations=['relu'] * len(hidden_sizes) + ['softmax'],
    loss='categorical_cross_entropy',
    weight_init='he',
)

# Training
print("\nFFNN")
start_time = time.time()
ffnn_history = ffnn_model.train(
    X_train, y_train_one_hot,
    batch_size=32,
    learning_rate=0.01,
    epochs=10,
    verbose=0
)
ffnn_training_time = time.time() - start_time
print(f"Custom FFNN training time: {ffnn_training_time:.2f} seconds")

# Prediksi
ffnn_predictions = ffnn_model.predict(X_test)
ffnn_pred_classes = np.argmax(ffnn_predictions, axis=1)
ffnn_accuracy = accuracy_score(y_test, ffnn_pred_classes)
print("Akurasi FFNN: " + str(ffnn_accuracy))

# Visualisasi
# ffnn_model.plot_loss(history=ffnn_history)
# ffnn_model.plot_weight_distribution()
# ffnn_model.plot_gradient_distribution()


FFNN
Custom FFNN training time: 13.34 seconds
Akurasi FFNN: 0.9637857142857142
